In [12]:
# Cell 1: Imports and setup
import numpy as np
from optint.data import synthetic_instance
from optint.test import test_bayesian_active, test_active
from optint.utils import learn_dag, compute_shd

# Set up a small test problem
p = 5  # Small for fast testing
seed = 42

problem = synthetic_instance(
    nnodes=p,
    DAG_type="random",
    sigma_square=0.4 * np.ones(p),
    a_size=p,
    a_target_nodes=None,
    std=False,
    seed=seed,
)

# Rescale target
a_star_true = np.linalg.solve(np.eye(p) - problem.B, problem.mu_target)
scale_factor = np.linalg.norm(a_star_true)
problem.mu_target = problem.mu_target / scale_factor

print(f"True DAG: {problem.DAG}")
print(f"Number of edges: {len(problem.DAG.arcs)}")

True DAG: [4][2][0][3|0][1|3]
Number of edges: 2


In [13]:
# Cell 2: Generate initial observational data and learn initial DAG
X_obs = problem.sample(a=np.zeros((p, 1)), n=50)
G_init = learn_dag(X_obs, p)

print(f"Initial DAG: {G_init}")
print(f"Initial SHD: {compute_shd(G_init, problem.DAG)}")

Initial DAG: [4][2][1][0][3|0,1]
Initial SHD: 2


In [14]:
# Cell 3: Set up options and run test
from optint.utils import compute_shd

class Opts:
    pass

opts = Opts()
opts.T = 10  # Short for testing
opts.W = 2   # Minimal warm-up
opts.n = 10
opts.known_noise = True
opts.measure = 'unif'
opts.acq = 'civ'

print("Running Bayesian active learning with iBGe (K=1)...")
print("="*60)

A_bayesian, Prob_bayesian, SHD_bayesian = test_bayesian_active(
    problem,
    G_init,
    opts,
    use_ibge=True,  # Enable iBGe
    K=1,            # Single DAG for now
    am=0.1          # iBGe hyperparameter
)

print(f"\n{'='*60}")
print(f"Initial SHD: {SHD_bayesian[0]}")
print(f"Final SHD:   {SHD_bayesian[-1]}")
print(f"SHD trajectory: {SHD_bayesian}")

Running Bayesian active learning with iBGe (K=1)...

Initial SHD: 2
Final SHD:   2
SHD trajectory: [2, 2, 2, 1, 2, 2, 2, 2, 2, 2]


In [15]:
# Cell 4: Compare with oracle (known DAG)
print("\nRunning oracle (known true DAG) for comparison...")
A_oracle, Prob_oracle = test_active(problem, opts)

# Compute distances to target
def get_distance(a, a_target):
    return np.linalg.norm(a - a_target)

dist_bayesian = [get_distance(a, problem.a_target) for a in A_bayesian]
dist_oracle = [get_distance(a, problem.a_target) for a in A_oracle]

print(f"\nFinal distance (Bayesian): {dist_bayesian[-1]:.3f}")
print(f"Final distance (Oracle):   {dist_oracle[-1]:.3f}")


Running oracle (known true DAG) for comparison...

Final distance (Bayesian): 0.944
Final distance (Oracle):   1.129
